#Data Wrangling with Pandas

[Aj. NesT the Series](http://bit.ly/ajnesttheseriesSubscribe)

Referenence: [Pyhton Data Cleaning Cookbook 2020](https://github.com/PacktPublishing/Python-Data-Cleaning-Cookbook)

##Practice 3: สร้างฟังก์ชันและคลาสที่กำหนดโดยผู้ใช้เพื่อทำความสะอาดข้อมูลแบบอัตโนมัติ

--> User-Defined Functions and Classes to Automate Data Cleaning

[Python Pandas Cheat Sheets](https://www.enthought.com/wp-content/uploads/Enthought-Python-Pandas-Cheat-Sheets-1-8-v1.0.2.pdf)

##Workshop 12: สร้างฟังก์ชันแสดง Summary Statistics และ Frequencies
--> Functions for displaying summary statistics and frequencies

###Input Datasets

* **[nls97f.csv](https://drive.google.com/file/d/1t9YHxkeCr4CUQqy8XDDu_RKM9uMKr0BP/view?usp=sharing)** - The survey started with a cohort
of individuals in 1997 who were born between 1980 and 1985, with annual
follow-ups each year through 2017. the National Longitudinal Survey (NLS) 1997-2017. 

####Question 12: สร้างฟังก์ชัน getmissings, gettots, makefreqs และ getcnts เพิ่มเติมต้องเขียนอย่างไรบ้าง?

**STEP 1:** นำเข้า Datasets โดยทำการ Upload ไฟล์ nls97f.csv ขึ้น Google Colab (ไฟล์นี้จะถูกลบเมื่อเราหลุดจาก Session หรือปิด Browser) หรือ Python IDE เช่น PyCharm, VSCode, Juputer Notebook, etc.

**STEP 2:** เขียนโปรแกรมเพื่อสร้างฟังก์ชันต่าง ๆ รวม 6 functions

In [4]:
#Define Functions
#(1) get first look
def getfirstlook(df, nrows=5, uniqueids=None):
  out = {}
  out['head'] = df.head(nrows)
  out['dtypes'] = df.dtypes
  out['nrows'] = df.shape[0]
  out['ncols'] = df.shape[1]
  out['index'] = df.index
  if (uniqueids is not None):
    out['uniqueids'] = df[uniqueids].nunique()
  return out

#(2) displaydict
def displaydict(dicttodisplay):
  print(*(': '.join(map(str, x)) \
    for x in dicttodisplay.items()), sep='\n\n')

#(3) get summary statistics
def gettots(df):
  out = {}
  out['min'] = df.min()
  out['per15'] = df.quantile(0.15)
  out['qr1'] = df.quantile(0.25)
  out['med'] = df.median()
  out['qr3'] = df.quantile(0.75)
  out['per85'] = df.quantile(0.85)
  out['max'] = df.max()
  out['count'] = df.count()
  out['mean'] = df.mean()
  out['iqr'] = out['qr3']-out['qr1']
  return pd.DataFrame(out)

#(4) count missings by columns and rows
def getmissings(df, byrowperc=False):
  return df.isnull().sum(),\
    df.isnull().sum(axis=1).value_counts(normalize=byrowperc).sort_index()

#(5) do frequencies and percentages for all category variables in data frame
def makefreqs(df, outfile):
  freqout = open(outfile, 'w') 
  for col in df.select_dtypes(include=["category"]):
    print(col, "----------------------", "frequencies",
    df[col].value_counts().sort_index(),"percentages",
    df[col].value_counts(normalize=True).sort_index(),
    sep="\n\n", end="\n\n\n", file=freqout)

  freqout.close()

#(6) get counts by groupings
def getcnts(df, cats, rowsel=None):
  tots = cats[:-1]
  catcnt = df.groupby(cats).size().reset_index(name='catcnt')
  totcnt = df.groupby(tots).size().reset_index(name='totcnt')
  percs = pd.merge(catcnt, totcnt, left_on=tots, 
    right_on=tots, how="left")
  percs['percent'] = percs.catcnt / percs.totcnt
  if (rowsel is not None):
    percs = percs.loc[eval("percs." + rowsel)]
  return percs

**STEP 3:** นำเข้า pandas librariey มาใช้งาน

**STEP 4:** ทำการอ่านไฟล์ .csv ด้วย pandas เก็บเป็น Dataframe

ใช้คำสั่ง .read_csv()

**STEP 5:** ตั้งค่าและแสดง index และ size ของ nls97f data

ใช้คำสั่ง set_index()

In [5]:
#import pandas library
import pandas as pd
nls97 = pd.read_csv("/content/nls97f.csv")
print(nls97)
nls97.set_index('personid', inplace=True)
print(nls97)

      personid  gender  birthmonth  ...      colenrfeb17      colenroct17 originalid
0       100061  Female           5  ...  1. Not enrolled  1. Not enrolled       8245
1       100139    Male           9  ...  1. Not enrolled  1. Not enrolled       3962
2       100284    Male          11  ...  1. Not enrolled  1. Not enrolled       3571
3       100292    Male           4  ...              NaN              NaN       2979
4       100583    Male           1  ...  1. Not enrolled  1. Not enrolled       8511
...        ...     ...         ...  ...              ...              ...        ...
8979    999291  Female           4  ...  1. Not enrolled  1. Not enrolled       7217
8980    999406    Male           7  ...  1. Not enrolled  1. Not enrolled          2
8981    999543  Female           8  ...  1. Not enrolled  1. Not enrolled       5113
8982    999698  Female           5  ...  1. Not enrolled  1. Not enrolled       7815
8983    999963  Female           9  ...  1. Not enrolled  1. Not 

**STEP 6:** เรียกใช้ฟังก์ชัน gettots() เพื่อแสดงค่า summary statistcs

In [6]:
#show summary statistics for continuous variables
print(gettots(nls97[['satverbal','satmath']]).T)
print()
print(gettots(nls97.filter(like="weeksworked")))  #Filter เอาเฉพาะ columns ที่มีชื่อคำว่า weeksworked

        satverbal      satmath
min      14.00000     7.000000
per15   390.00000   390.000000
qr1     430.00000   430.000000
med     500.00000   500.000000
qr3     570.00000   580.000000
per85   620.00000   621.000000
max     800.00000   800.000000
count  1406.00000  1407.000000
mean    499.72404   500.590618
iqr     140.00000   150.000000

               min  per15   qr1   med  ...   max  count       mean   iqr
weeksworked00  0.0    0.0   5.0  26.0  ...  53.0   8603  26.417761  45.0
weeksworked01  0.0    0.0  10.0  33.0  ...  52.0   8564  29.784096  41.0
weeksworked02  0.0    0.0  13.0  38.0  ...  52.0   8556  31.805400  39.0
weeksworked03  0.0    0.0  14.0  43.0  ...  52.0   8490  33.469611  38.0
weeksworked04  0.0    1.0  18.0  46.0  ...  52.0   8458  35.104635  34.0
weeksworked05  0.0    5.0  22.0  50.0  ...  53.0   8403  37.316435  31.0
weeksworked06  0.0    9.0  27.0  51.0  ...  52.0   8340  38.429976  25.0
weeksworked07  0.0   10.0  30.0  52.0  ...  52.0   8272  39.241296  22.0
w

**STEP 7:** เรียกใช้ฟังก์ชัน getmissings() เพื่อนับจำนวน missing data ต่อ column และต่อ row

In [16]:
#count missing per column and per row
missingsbycols, missingsbyrows = getmissings(nls97[['weeksworked16','weeksworked17']])
print(missingsbycols)
print()
print(missingsbyrows)

weeksworked16    1916
weeksworked17    2314
dtype: int64

0    6641
1     456
2    1887
dtype: int64


**STEP 8:** เรียกใช้ฟังก์ชัน makefreqs เพื่อรับความถี่ของ categorical columns

In [18]:
#do frequencies for categorical columns
#แปลง data types จาก object ให้เป็น category
nls97.loc[:, nls97.dtypes == 'object'] = \
  nls97.select_dtypes(['object']). \
  apply(lambda x: x.astype('category'))
#เรียกใช้ฟังก์ชัน makefreqs เขียนไฟล์แสดงจำนวนความถี่ที่นับได้
makefreqs(nls97, "/content/nlsfreqs.txt")

**STEP 9:** เรียกใช้ฟังก์ชัน getcnts นับจำนวนและ % ตาม groups

In [19]:
#do counts and percentages by groups
print(getcnts(nls97, ['maritalstatus','gender','colenroct00']))
print()
print(getcnts(nls97, ['maritalstatus','gender','colenroct00'], "colenroct00.str[0:1]=='1'"))

    maritalstatus  gender         colenroct00  catcnt  totcnt   percent
0        Divorced  Female     1. Not enrolled     317     393  0.806616
1        Divorced  Female  2. 2-year college       35     393  0.089059
2        Divorced  Female   3. 4-year college      41     393  0.104326
3        Divorced    Male     1. Not enrolled     238     270  0.881481
4        Divorced    Male  2. 2-year college       15     270  0.055556
5        Divorced    Male   3. 4-year college      17     270  0.062963
6         Married  Female     1. Not enrolled    1168    1636  0.713936
7         Married  Female  2. 2-year college      143    1636  0.087408
8         Married  Female   3. 4-year college     325    1636  0.198655
9         Married    Male     1. Not enrolled    1094    1430  0.765035
10        Married    Male  2. 2-year college       93    1430  0.065035
11        Married    Male   3. 4-year college     243    1430  0.169930
12  Never-married  Female     1. Not enrolled    1094    1307  0